In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

engine = create_engine("sqlite:///userdata.db", echo=True)

In [2]:
from sqlalchemy.orm import DeclarativeBase

class Base(DeclarativeBase):
    pass

In [3]:
from sqlalchemy import String, DateTime, func
from sqlalchemy.orm import mapped_column

class Userdata(Base):
    __tablename__ = "userdata"
    user_id = mapped_column(String(32), primary_key=True, nullable=False)
    user_data = mapped_column(String(640))
    time_stamp = mapped_column(DateTime(timezone=True), server_default=func.now(), onupdate=func.now())

In [ ]:
from sqlalchemy import String, DateTime
from sqlalchemy.orm import mapped_column

class Userdata(Base):
    __tablename__ = "userdata"
    user_id = mapped_column(String(32), primary_key=True, nullable=False)
    user_data = mapped_column(String(640))
    time_stamp = mapped_column(DateTime(timezone=True))

In [ ]:
print(Base.metadata.tables)

In [ ]:
print(Userdata)

Database is created after the step below. See userdata.db file in folder

In [4]:
Base.metadata.create_all(engine)

2023-10-19 08:44:33,425 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-19 08:44:33,429 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("userdata")
2023-10-19 08:44:33,430 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-19 08:44:33,432 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("userdata")
2023-10-19 08:44:33,433 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-19 08:44:33,433 INFO sqlalchemy.engine.Engine 
CREATE TABLE userdata (
	user_id VARCHAR(32) NOT NULL, 
	user_data VARCHAR(640), 
	time_stamp DATETIME DEFAULT (CURRENT_TIMESTAMP), 
	PRIMARY KEY (user_id)
)


2023-10-19 08:44:33,436 INFO sqlalchemy.engine.Engine [no key 0.00160s] ()
2023-10-19 08:44:33,445 INFO sqlalchemy.engine.Engine COMMIT


In section below, need to manually commit and close session

In [5]:
from datetime import datetime

session = Session(engine)
user = Userdata(user_id="Brad", user_data="Data from brad")
session.add(user)
user = Userdata(user_id="Larry", user_data="More data")
session.add(user)
user = Userdata(user_id="Jane", user_data="Even more data")
session.add(user)
session.commit()
session.close()

2023-10-19 08:44:53,170 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-19 08:44:53,173 INFO sqlalchemy.engine.Engine INSERT INTO userdata (user_id, user_data) VALUES (?, ?), (?, ?), (?, ?) RETURNING time_stamp, user_id
2023-10-19 08:44:53,174 INFO sqlalchemy.engine.Engine [generated in 0.00015s (insertmanyvalues) 1/1 (ordered)] ('Brad', 'Data from brad', 'Larry', 'More data', 'Jane', 'Even more data')
2023-10-19 08:44:53,179 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
from sqlalchemy import update

def db_update(session, id, data):
    stmt = (update(Userdata)
            .where(Userdata.user_id == id)
            .values(user_data=data))
    session.execute(stmt)

In [ ]:
db_update(session, "Brad", "New New New Bradley")
user = Userdata(user_id="Sally", user_data="Sally data")
session.add(user)
session.commit()

In [ ]:
from datetime import datetime

session = Session(engine)
user = Userdata(user_id="Brad", user_data="Data from brad", time_stamp=datetime.now())
session.add(user)
user = Userdata(user_id="Larry", user_data="More data", time_stamp=datetime.now())
session.add(user)
user = Userdata(user_id="Jane", user_data="Even more data", time_stamp=datetime.now())
session.add(user)
session.commit()
session.close()

if you just use Session class, conext manager does not automatically write data to DB at end of block, or rollback if error. need to call *session.commit()* before end of block


In [ ]:
with Session(engine) as session:
    user = Userdata(user_id="Allan", user_data="Data from Allan", time_stamp=datetime.now())
    session.add(user)
    user = Userdata(user_id="Chad", user_data="More Chad data", time_stamp=datetime.now())
    session.add(user)
    user = Userdata(user_id="Mary", user_data="Even more Mary data", time_stamp=datetime.now())
    session.add(user)
    session.commit()


In the section below, the sessiomaker's *begin* method creates a context manager that will run purge() and commit() at the end of the block so they do not need to be explixitly called using *session.commit()*. Also takes care of rollback if write fails. See https://docs.sqlalchemy.org/en/14/orm/session_basics.html#getting-a-session for more ways to do this

In [ ]:
from sqlalchemy.orm import sessionmaker

Session2 = sessionmaker(engine)

with Session2.begin() as session:
    user1 = Userdata(user_id="xx", user_data="Data from Phil", time_stamp=datetime.now())
    user2 = Userdata(user_id="yy", user_data="More Karen data", time_stamp=datetime.now())
    user3 = Userdata(user_id="ss", user_data="Gary data", time_stamp=datetime.now())
    session.add_all([user1, user2, user3])

experiment with flush

In [43]:
session3.close()
from sqlalchemy import select

session3 = Session(engine)
session3.autoflush=False

user = Userdata(user_id="Flusher6", user_data="Flushed6")
session3.add(user)


2023-10-19 09:13:49,643 INFO sqlalchemy.engine.Engine ROLLBACK


In [47]:
session3.autoflush=True
f = session3.get(Userdata, "Flusher6")

2023-10-19 09:21:16,186 INFO sqlalchemy.engine.Engine INSERT INTO userdata (user_id, user_data) VALUES (?, ?) RETURNING time_stamp
2023-10-19 09:21:16,188 INFO sqlalchemy.engine.Engine [cached since 2122s ago] ('Flusher6', 'Flushed6')
2023-10-19 09:21:16,195 INFO sqlalchemy.engine.Engine SELECT userdata.user_id AS userdata_user_id, userdata.user_data AS userdata_user_data, userdata.time_stamp AS userdata_time_stamp 
FROM userdata 
WHERE userdata.user_id = ?
2023-10-19 09:21:16,196 INFO sqlalchemy.engine.Engine [cached since 535.6s ago] ('Flusher6',)


In [49]:
print(f)
print(f.user_id, f.user_data, f.time_stamp)

Flusher6 Flushed6 2023-10-19 13:21:16


In [36]:
stmt = select(Userdata)
results = session3.execute(stmt).fetchall()
print(results)
for x in results:
    print(type(x))
    # print(x.user_id, x.user_data, x.time_stamp)

2023-10-19 09:06:53,609 INFO sqlalchemy.engine.Engine SELECT userdata.user_id, userdata.user_data, userdata.time_stamp 
FROM userdata
2023-10-19 09:06:53,610 INFO sqlalchemy.engine.Engine [cached since 972.2s ago] ()
[(<__main__.Userdata object at 0x000002075A1C8C10>,), (<__main__.Userdata object at 0x000002075A1D1610>,), (<__main__.Userdata object at 0x000002075A1D3210>,), (<__main__.Userdata object at 0x000002075A1D2490>,), (<__main__.Userdata object at 0x000002075A1D0E50>,), (<__main__.Userdata object at 0x000002075A0E2550>,)]
<class 'sqlalchemy.engine.row.Row'>
<class 'sqlalchemy.engine.row.Row'>
<class 'sqlalchemy.engine.row.Row'>
<class 'sqlalchemy.engine.row.Row'>
<class 'sqlalchemy.engine.row.Row'>
<class 'sqlalchemy.engine.row.Row'>


In [54]:
stmt = select(Userdata)
results = session3.execute(stmt).scalars()
for x in results:
    print(x.user_id, x.user_data, x.time_stamp)

2023-10-19 11:44:33,915 INFO sqlalchemy.engine.Engine SELECT userdata.user_id, userdata.user_data, userdata.time_stamp 
FROM userdata
2023-10-19 11:44:33,916 INFO sqlalchemy.engine.Engine [cached since 9690s ago] ()
Brad Data from brad 2023-10-19 12:44:53
Larry More data 2023-10-19 12:44:53
Jane Even more data 2023-10-19 12:44:53
Flusher Flushed? 2023-10-19 12:45:54
Flusher3 Flushed33 2023-10-19 12:53:46
Flusher6 Flushed6 2023-10-19 13:21:16


In [55]:
session3.commit()

2023-10-19 11:53:43,330 INFO sqlalchemy.engine.Engine COMMIT


In [61]:
d1 = session3.execute(select(Userdata).where(Userdata.user_id=="Flusher")).scalar()
d2 = session3.execute(select(Userdata).where(Userdata.user_id=="Flusher3")).scalar()
d3 = session3.execute(select(Userdata).where(Userdata.user_id=="Flusher6")).scalar()
session3.delete(d1)
session3.delete(d2)
session3.delete(d3)
session3.commit()

2023-10-19 11:59:21,140 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-19 11:59:21,142 INFO sqlalchemy.engine.Engine SELECT userdata.user_id, userdata.user_data, userdata.time_stamp 
FROM userdata 
WHERE userdata.user_id = ?
2023-10-19 11:59:21,143 INFO sqlalchemy.engine.Engine [generated in 0.00107s] ('Flusher',)
2023-10-19 11:59:21,145 INFO sqlalchemy.engine.Engine SELECT userdata.user_id, userdata.user_data, userdata.time_stamp 
FROM userdata 
WHERE userdata.user_id = ?
2023-10-19 11:59:21,146 INFO sqlalchemy.engine.Engine [cached since 0.004878s ago] ('Flusher3',)
2023-10-19 11:59:21,148 INFO sqlalchemy.engine.Engine SELECT userdata.user_id, userdata.user_data, userdata.time_stamp 
FROM userdata 
WHERE userdata.user_id = ?
2023-10-19 11:59:21,149 INFO sqlalchemy.engine.Engine [cached since 0.008344s ago] ('Flusher6',)
2023-10-19 11:59:21,154 INFO sqlalchemy.engine.Engine DELETE FROM userdata WHERE userdata.user_id = ?
2023-10-19 11:59:21,155 INFO sqlalchemy.engine.Engine [g

In [62]:
stmt = select(Userdata).where(Userdata.user_id == "Brad")
brad = session.execute(stmt).scalar()
brad.user_data = "Changed data"
session.commit()

2023-10-19 13:13:54,847 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-19 13:13:54,849 INFO sqlalchemy.engine.Engine SELECT userdata.user_id, userdata.user_data, userdata.time_stamp 
FROM userdata 
WHERE userdata.user_id = ?
2023-10-19 13:13:54,850 INFO sqlalchemy.engine.Engine [cached since 4474s ago] ('Brad',)
2023-10-19 13:13:54,853 INFO sqlalchemy.engine.Engine UPDATE userdata SET user_data=?, time_stamp=CURRENT_TIMESTAMP WHERE userdata.user_id = ?
2023-10-19 13:13:54,854 INFO sqlalchemy.engine.Engine [generated in 0.00146s] ('Changed data', 'Brad')
2023-10-19 13:13:54,857 INFO sqlalchemy.engine.Engine COMMIT


In [64]:
from sqlalchemy import update

session.execute(
    update(Userdata)
    .where(Userdata.user_id == "Brad")
    .values(user_data="Changed again")
)

2023-10-19 13:30:45,450 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-19 13:30:45,452 INFO sqlalchemy.engine.Engine UPDATE userdata SET user_data=?, time_stamp=CURRENT_TIMESTAMP WHERE userdata.user_id = ?
2023-10-19 13:30:45,452 INFO sqlalchemy.engine.Engine [generated in 0.00078s] ('Changed again', 'Brad')


In [65]:
session.commit()

2023-10-19 13:31:19,962 INFO sqlalchemy.engine.Engine COMMIT


In [68]:
id = "Brian"
brad = session.execute(
    select(Userdata).where(Userdata.user_id == id)
    ).scalar()
if brad:
    brad.user_data = "More changes data"
    session.commit()
else:
    print(f"User '{id}' does not exist")
session.close()

2023-10-19 13:37:22,353 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-19 13:37:22,355 INFO sqlalchemy.engine.Engine SELECT userdata.user_id, userdata.user_data, userdata.time_stamp 
FROM userdata 
WHERE userdata.user_id = ?
2023-10-19 13:37:22,356 INFO sqlalchemy.engine.Engine [cached since 5881s ago] ('Brian',)
User 'Brian' does not exist
2023-10-19 13:37:22,358 INFO sqlalchemy.engine.Engine ROLLBACK


In [71]:
id = "Brad"
brad = session.execute(
    select(Userdata).where(Userdata.user_id == id)
    ).scalar()
if brad:
    session.execute(
        update(Userdata)
        .where(Userdata.user_id == id)
        .values(user_data="Changed yet again")
    )
    session.commit()
else:
    print(f"User '{id}' does not exist")
session.close()

2023-10-19 13:50:59,384 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-19 13:50:59,384 INFO sqlalchemy.engine.Engine SELECT userdata.user_id, userdata.user_data, userdata.time_stamp 
FROM userdata 
WHERE userdata.user_id = ?
2023-10-19 13:50:59,385 INFO sqlalchemy.engine.Engine [cached since 6698s ago] ('Brad',)
2023-10-19 13:50:59,386 INFO sqlalchemy.engine.Engine UPDATE userdata SET user_data=?, time_stamp=CURRENT_TIMESTAMP WHERE userdata.user_id = ?
2023-10-19 13:50:59,388 INFO sqlalchemy.engine.Engine [cached since 1214s ago] ('Changed yet again', 'Brad')
2023-10-19 13:50:59,391 INFO sqlalchemy.engine.Engine COMMIT


End experiment with flush

In [ ]:
from sqlalchemy import select

session = Session(engine)

user = Userdata(user_id="Flusher", user_data="Flushed?")
session.add(user)
stmt = select(Userdata).execution_options(autoflush=False)
results = session.execute(stmt).scalars()
for x in results:
    print(x.user_id, x.user_data, x.time_stamp)

In [ ]:
from sqlalchemy import select

stmt = select(Userdata)
results = session.execute(stmt).scalars()
for x in results:
    print(x.user_id)

In [ ]:
session.flush()

In [ ]:
stmt = select(Userdata).execution_options(autoflush=False)
results = session.execute(stmt).fetchall()
for x in results:
    print(type(x[0]))

In [ ]:
stmt = select(Userdata).execution_options(autoflush=False)
results = session.execute(stmt).scalars()
for x in results:
    print(x.user_data)

In [ ]:
stmt = select(Userdata.user_data, Userdata.time_stamp).where(Userdata.user_id=="Brad")
results = session.execute(stmt).one()
print(results)
print(results.time_stamp)

In [ ]:
session.close()